In [1]:
# -*- coding: utf-8 -*-

import netCDF4 as nc4
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [2]:
""" ********  FILES TO PLOT  *************
    *****************************************
    tenseur divrot - freeslip
    convergence 1°/4, /8, /16, /48
    0°, 45°
"""

dirt = "/gpfswork/rech/bho/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/article_AM98/"
dirm = "/gpfswork/rech/bho/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/article_AM98/MESHMASK/"

#ListA = [[4 , 0. , dirt+"EXP_4_ens_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                   dirm+"mesh_mask_n4_t0.nc"], 
#         [8 , 0. , dirt+"EXP_8_ens_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                   dirm+"mesh_mask_n8_t0.nc"], 
#         [16 , 0. ,dirt+"EXP_16_ens_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                   dirm+"mesh_mask_n16_t0.nc"],  
#         [32 , 0. ,dirt+"EXP_32_ens_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                   dirm+"mesh_mask_n32_t0.nc"],
#         [48 , 0. ,dirt+"EXP_48_ens_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                   dirm+"mesh_mask_n48_t0.nc"]] 
#
#ListB = [[4 , 45. , dirt+"EXP_4_ens_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                    dirm+"mesh_mask_n4_t45.nc"], 
#         [8 , 45. , dirt+"EXP_8_ens_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                    dirm+"mesh_mask_n8_t45.nc"], 
#         [16 , 45. ,dirt+"EXP_16_ens_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                    dirm+"mesh_mask_n16_t45.nc"],  
#         [32 , 45. ,dirt+"EXP_32_ens_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                    dirm+"mesh_mask_n32_t45.nc"],
#         [48 , 45. ,dirt+"EXP_48_ens_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
#                    dirm+"mesh_mask_n48_t45.nc"]] 

ListA = [[4 , 0. , dirt+"EXP_4_ens_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n4_t0.nc", 60], 
         [8 , 0. , dirt+"EXP_8_ens_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n8_t0.nc", 60], 
         [16 , 0. ,dirt+"EXP_16_ens_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n16_t0.nc", 60],  
         [32 , 0. ,dirt+"EXP_32_ens_fs0_rot/AM98_1m_00010101_00251230_grid_T.nc",
                   dirm+"mesh_mask_n32_t0.nc", 60]] 

ListB = [[4 , 45. , dirt+"EXP_4_ens_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
                    dirm+"mesh_mask_n4_t45.nc", 60], 
         [8 , 45. , dirt+"EXP_8_ens_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
                    dirm+"mesh_mask_n8_t45.nc", 60], 
         [16 , 45. ,dirt+"EXP_16_ens_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
                    dirm+"mesh_mask_n16_t45.nc", 60],  
         [32 , 45. ,dirt+"EXP_32_ens_fs45_rot/AM98_1m_00010101_00251230_grid_T.nc",
                    dirm+"mesh_mask_n32_t45.nc", 60]] 

Listdt = [ListA, ListB]

""" *************************** PARAMETERS
"""

NI = 4 ; NJ = 2
L=2000E3 # 2000 km
vmin = 200 ; vmax = 800
tickszer = [np.float64(i)*L/4. for i in range(5)]
dz=100./3.
N = (vmax-vmin)/dz
cticks = np.arange(200, 800+dz, 100.) # 3 x 100/3
palette = plt.get_cmap('RdBu_r', N)

save = 1 ; psave = "figure3.png" ; dpi = 800


In [1]:
fig, ax = plt.subplots(NI,NJ, figsize=(NJ*3,NI*3), dpi = dpi, squeeze=False)

""" *************************** DATA
"""
for i in range(NI):
    for j in range(NJ):
        n,theta,pdtT,pmm,nmean = Listdt[j][i]
        try :
            dt = nc4.Dataset(pdtT)
            mm = nc4.Dataset(pmm)
        except :
            ax[i][j].axis('off')
            print("... dataframe[%d,%d], missing : %s" % (i,j,pdtT))
            continue
        dx = 100E3/np.float64(n)
        # reading
        print("... dataframe[%d,%d], n = %d : %s" % (i,j,n, pdtT))
        tmask = mm.variables['tmask'][0,0]
        glamt = mm.variables['glamt'][0] ; gphit = mm.variables['gphit'][0]
        # grid
        lx = dx*np.cos((45+theta)*np.pi/180)/np.sqrt(2) ; ly = dx*np.sin((45+theta)*np.pi/180)/np.sqrt(2)
        nx,ny = np.shape(glamt)
        gridx = np.zeros((nx,ny)) ; gridy = np.zeros((nx,ny))
        gridx = glamt - lx ; gridy = gphit - ly
        # data
        if nmean>0:
            ssh = dt.variables['sshmeaned'][-nmean:,:,:].copy() +500.
            data = np.mean(ssh[:,:,:], axis=0)
            data = np.ma.masked_where(tmask==0,data)
        else :
            ssh = dt.variables['sshmeaned'][-1,:,:].copy() +500.
            data = np.ma.masked_where(tmask==0,ssh)
        dt.close()
        #nT = np.shape(dt.variables['sshmeaned'])[0]
        #print("nT = %d" % nT)
        # plot
        cf = ax[i][j].pcolormesh(gridx, gridy, data,
                            vmin=vmin, vmax=vmax, alpha = 1.,
                            # levels = levels,
                            cmap = palette)
        # ax[i][j].title.set_text(titles[i][j])
        # contour
        c1= ax[i][j].contour(glamt,gphit, data,
                        levels = cticks,
                        vmin=vmin,vmax=vmax,
                        linewidths =0.8, colors=('k',),linestyles = "solid")
        c1= ax[i][j].contour(glamt,gphit, data,
                        levels = [500],
                        vmin=vmin,vmax=vmax,
                        linewidths =1.4, colors=('k',),linestyles = "solid")
        #ax[i][j].clabel(c1, fmt='%3.0f', colors='k', fontsize=6)
        
        
""" *************************** PRETTY
"""
for i in range(NI):
    for j in range(NJ):
        try :
            n,theta,pdtT,pmm,_ = Listdt[j][i]
        except :
            continue
        dx = 100E3/4.
        limx = dx ; limy = dx
        # print(tickszer)
        ax[i][j].set_xlim(-limx,L + limx)
        ax[i][j].set_xticks(tickszer)
        ax[i][j].set_ylim(-limy,L + limy)
        ax[i][j].set_yticks(tickszer)
        ax[i][j].set_xticklabels([]) ; ax[i][j].set_yticklabels([])
        #
        #if (i==NI-1):
        #    ax[i][j].set_xticklabels(["%1.1f" % (x/1E6) for x in tickszer])
        #    ax[i][j].set_xlabel("X (1000 km)", fontsize = 6)
        #
        #if (j==0):
        #    ax[i][j].set_yticklabels(["%1.1f" % (x/1E6) for x in tickszer])
        #    ax[i][j].set_ylabel("Y (1000 km)", fontsize = 6)
        #
        ax[i][j].patch.set_color('0.')
        ax[i][j].xaxis.set_minor_locator(AutoMinorLocator())
        ax[i][j].yaxis.set_minor_locator(AutoMinorLocator())
        ax[i][j].tick_params(axis = "both", which = 'both', width=1., labelsize = 5, pad = 3, \
                             bottom = True, top = True, left = True, right = True)
        ax[i][j].tick_params(which='minor',length = 3)
        ax[i][j].tick_params(which='major',length = 5)
        ax[i][j].set_aspect(aspect='equal') # data coordinate 'equal'

#plt.subplots_adjust(left=0, bottom=0, right=1, top=1, \
#                    wspace=-0.15, hspace=0.1)
plt.subplots_adjust(left=0.01, bottom=0.01, right=0.99, top=0.99, \
                    wspace=0.04, hspace=0.08) # h is height of L1 (NJ) and w is width of L2 (NI)

""" *************************** COLORBAR
"""
#cbar=plt.colorbar(cf, ax=ax[:], shrink=0.8, location = "bottom",
#                  aspect = 30, fraction=0.05,  extend = 'both', pad = 0.02) #
#cbar.set_ticks(cticks)
#cbar.set_ticklabels(["%dm" % s for s in cticks])
#cbar.ax.tick_params(size=0)
#for _ in cticks :
#    cbar.ax.axvline(x=_, c='black', lw = 0.5)
#cbar.ax.axvline(x=[500], c='black', lw = 0.5)


""" *************************** SAVING
"""

if save :
    print("\nsaving : %s" % psave)
    fig.savefig(psave, dpi = dpi)
    plt.close()
    print("figure closed")
plt.show()




NameError: name 'plt' is not defined